In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.signal import butter, lfilter, iirnotch, filtfilt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import joblib

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mtcaic3/sample_submission.csv
/kaggle/input/mtcaic3/README.md
/kaggle/input/mtcaic3/validation.csv
/kaggle/input/mtcaic3/train.csv
/kaggle/input/mtcaic3/test.csv
/kaggle/input/mtcaic3/SSVEP/validation/S32/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/validation/S33/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/validation/S31/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/validation/S35/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/validation/S34/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/test/S39/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/test/S40/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/test/S37/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/test/S38/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/test/S36/1/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/train/S26/7/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/train/S26/2/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/train/S26/5/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/train/S26/8/EEGdata.csv
/kaggle/input/mtcaic3/SSVEP/train/S26/3/EEGdata.csv
/kaggle/input/mtcaic3/SS

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def notch_filter(data, fs=250, freq=50.0, Q=30.0):
    b, a = iirnotch(freq / (0.5 * fs), Q)
    return filtfilt(b, a, data, axis=0)

def bandpass_filter(data, lowcut=7, highcut=30, fs=250):
    b, a = butter_bandpass(lowcut, highcut, fs)
    data = lfilter(b, a, data, axis=0)
    return notch_filter(data, fs=fs)
def add_noise(eeg, noise_std=0.01):
    noise = np.random.normal(0, noise_std, eeg.shape)
    return eeg + noise

def normalize_per_trial(eeg):
    mean = np.mean(eeg, axis=1, keepdims=True)
    std = np.std(eeg, axis=1, keepdims=True) + 1e-6
    return (eeg - mean) / std

def standardize_global(eeg, mean, std):
    return (eeg - mean) / (std + 1e-6)
   
def mixup_data(x, y, alpha=0.4):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [4]:
class EEGDataset(Dataset):
    def __init__(self, task, split, base_path='./kaggle/input/mtcaic3', apply_noise=True, apply_trial_norm=True, apply_global_std=False, global_mean=None, global_std=None):
        self.task = task
        self.split = split
        self.base_path = base_path
        self.apply_noise = apply_noise
        self.apply_trial_norm = apply_trial_norm
        self.apply_global_std = apply_global_std
        self.global_mean = global_mean
        self.global_std = global_std

        self.meta_df = pd.read_csv(os.path.join(base_path, f'{split}.csv'))
        self.meta_df = self.meta_df[self.meta_df['task'] == task]

        self.label_encoder = LabelEncoder()
        if split != 'test':
            self.meta_df['label_enc'] = self.label_encoder.fit_transform(self.meta_df['label'])

    def __len__(self):
        return len(self.meta_df)

    def __getitem__(self, idx):
        row = self.meta_df.iloc[idx]
        eeg_path = os.path.join(
            self.base_path,
            row['task'],
            self.split,
            row['subject_id'],
            str(row['trial_session']),
            'EEGdata.csv'
        )

        df = pd.read_csv(eeg_path)

        samples_per_trial = 2250 if row['task'] == 'MI' else 1750
        trial_num = int(row['trial'])
        start_idx = (trial_num - 1) * samples_per_trial
        end_idx = start_idx + samples_per_trial

        df = df.iloc[start_idx:end_idx]
        eeg = df[['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']].values
        #eeg = df[['C3', 'CZ', 'C4']].values

        eeg = bandpass_filter(eeg)

        # Apply per-trial z-score normalization
        if self.apply_trial_norm:
            eeg = normalize_per_trial(eeg)

        # Apply global standardization
        if self.apply_global_std and self.global_mean is not None and self.global_std is not None:
            eeg = standardize_global(eeg, self.global_mean, self.global_std)

        # Apply Gaussian noise
        if self.apply_noise and self.split == 'train':
            eeg = add_noise(eeg)

        eeg = eeg.T  # Shape: (channels, time)
        eeg = torch.tensor(eeg.copy(), dtype=torch.float32)  # .copy() to avoid negative strides

        if self.split != 'test':
            label = torch.tensor(row['label_enc'], dtype=torch.long)
            return eeg, label
        else:
            return eeg

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Swish activation
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# CBAM Block (Channel & Spatial Attention)
class CBAM(nn.Module):
    def __init__(self, channel, reduction=16):
        super(CBAM, self).__init__()
        self.channel_att = SEBlock(channel, reduction)
        self.spatial_att = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_out = self.channel_att(x)
        avg_out = torch.mean(x_out, dim=1, keepdim=True)
        max_out, _ = torch.max(x_out, dim=1, keepdim=True)
        concat = torch.cat([avg_out, max_out], dim=1)
        scale = self.spatial_att(concat)
        return x_out * scale

# Same SEBlock, configurable reduction
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=4):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Tiny Transformer Encoder for temporal modeling
class TemporalTransformer(nn.Module):
    def __init__(self, dim, heads=2, layers=1):
        super(TemporalTransformer, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=layers)

    def forward(self, x):
        # x: [B, C, 1, T] -> flatten spatial dims -> [B, T, C]
        b, c, _, t = x.size()
        x = x.squeeze(2).permute(0, 2, 1)
        x = self.transformer(x)
        # [B, T, C] -> [B, C, 1, T]
        x = x.permute(0, 2, 1).unsqueeze(2)
        return x

class EEGNetPro(nn.Module):
    def __init__(self, num_classes, channels=8, samples=2250):
        super(EEGNetPro, self).__init__()
        self.swish = Swish()

        self.firstconv = nn.Sequential(
            nn.Conv2d(1, 16, (1, 32), padding=(0, 32), bias=False),
            nn.BatchNorm2d(16)
        )

        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, (channels, 1), groups=16, bias=False),
            nn.BatchNorm2d(32),
            self.swish,
            nn.BatchNorm2d(32),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(0.5)
        )
        self.se_block1 = SEBlock(32, reduction=4)

        self.separableConv = nn.Sequential(
            nn.Conv2d(32, 32, (1, 16), padding=(0, 8), bias=False),
            nn.BatchNorm2d(32),
            self.swish,
            nn.BatchNorm2d(32),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(0.5)
        )

        self.cbam = CBAM(32)

        self.temporal_transformer = TemporalTransformer(dim=32, heads=2, layers=1)

        self.flatten_dim = self._get_flattened_size(channels, samples)
        self.classify = nn.Linear(self.flatten_dim, num_classes)

    def _get_flattened_size(self, channels, samples):
        with torch.no_grad():
            x = torch.zeros(1, 1, channels, samples)
            x = self.firstconv(x)
            x = self.depthwiseConv(x)
            x = self.se_block1(x)
            x = self.separableConv(x)
            x = self.cbam(x)
            x = self.temporal_transformer(x)
            return x.reshape(1, -1).shape[1]

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.firstconv(x)
        x = self.depthwiseConv(x)
        x = self.se_block1(x)
        x = self.separableConv(x)
        x = self.cbam(x)
        x = self.temporal_transformer(x)
        x = x.reshape(x.size(0), -1)
        return self.classify(x)

In [6]:
def train_one_epoch(model, dataloader, optimizer, criterion,device, alpha=0.4):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for X, y in tqdm(dataloader, desc='Training'):
        X, y = X.to(device), y.to(device)
        
        # Apply mixup
        mixed_X, y_a, y_b, lam = mixup_data(X, y, alpha)
        optimizer.zero_grad()
        outputs = model(mixed_X)

        # Mixup loss
        loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # For metrics: use clean inputs (no mixup) just for logging
        with torch.no_grad():
            preds_clean = torch.argmax(model(X), dim=1)
            all_preds.append(preds_clean.cpu().numpy())
            all_labels.append(y.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')

    return running_loss / len(dataloader), acc, f1


def validate(model, dataloader, criterion,device):
    model.eval()
    val_loss = 0
    preds, targets = [], []
    with torch.no_grad():
        for X, y in tqdm(dataloader, desc='Validation'):
            X, y = X.to(device), y.to(device)
            out = model(X)
            loss = criterion(out, y)
            val_loss += loss.item()
            preds.extend(out.argmax(1).cpu().numpy())
            targets.extend(y.cpu().numpy())
    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average='macro')
    return val_loss / len(dataloader), acc, f1
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        n_class = pred.size(1)
        log_preds = F.log_softmax(pred, dim=1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (n_class - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), 1.0 - self.smoothing)
        return torch.mean(torch.sum(-true_dist * log_preds, dim=1))

In [7]:
base_path = '/kaggle/input/mtcaic3'
train_ds = EEGDataset(task='SSVEP', split='train', base_path=base_path)
val_ds = EEGDataset(task='SSVEP', split='validation', base_path=base_path)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)
if torch.cuda.is_available():
    print("GPU")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGNetPro(num_classes = 4, samples=1750).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)
criterion = LabelSmoothingCrossEntropy(smoothing=0.1)

In [8]:
best_val_f1 = 0
for epoch in range(1, 31):
    print(f"\nEpoch {epoch}")
    train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion,device)
    val_loss, val_acc, val_f1 = validate(model, val_loader, criterion,device)

    print(f"Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}")
    print(f"Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'best_eegnet_model.pt')
        print("Best model updated.")


Epoch 1


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.04it/s]


Train Acc: 0.2729 | Train F1: 0.2712
Val Acc: 0.3000 | Val F1: 0.2579
Best model updated.

Epoch 2


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.13it/s]


Train Acc: 0.3304 | Train F1: 0.3302
Val Acc: 0.2400 | Val F1: 0.2158

Epoch 3


...

Train Acc: 0.7425 | Train F1: 0.7424
Val Acc: 0.4800 | Val F1: 0.4738

Epoch 29


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


Train Acc: 0.7467 | Train F1: 0.7471
Val Acc: 0.5400 | Val F1: 0.5382

Epoch 30


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]

Train Acc: 0.7462 | Train F1: 0.7457
Val Acc: 0.4800 | Val F1: 0.4666


In [ ]:
for epoch in range(1, 31):
    print(f"\nEpoch {epoch}")
    train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion,device)
    val_loss, val_acc, val_f1 = validate(model, val_loader, criterion,device)

    print(f"Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}")
    print(f"Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'best_eegnet_model.pt')
        print("Best model updated.")


Epoch 1


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


Train Acc: 0.7475 | Train F1: 0.7474
Val Acc: 0.5000 | Val F1: 0.4880

Epoch 2


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]


Train Acc: 0.7467 | Train F1: 0.7465
Val Acc: 0.3600 | Val F1: 0.3566

Epoch 3


...

Validation: 100%|██████████| 4/4 [00:03<00:00,  1.16it/s]


Train Acc: 0.7692 | Train F1: 0.7692
Val Acc: 0.4200 | Val F1: 0.4146

Epoch 12


Validation: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


Train Acc: 0.7704 | Train F1: 0.7704
Val Acc: 0.4800 | Val F1: 0.4670

Epoch 13


Training:  78%|███████▊  | 117/150 [02:34<00:43,  1.33s/it]

In [ ]:
for epoch in range(1, 31):
    print(f"\nEpoch {epoch}")
    train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion,device)
    val_loss, val_acc, val_f1 = validate(model, val_loader, criterion,device)

    print(f"Train Acc: {train_acc:.4f} | Train F1: {train_f1:.4f}")
    print(f"Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'best_eegnet_model.pt')
        print("Best model updated.")

In [ ]:
best_val_f1